## Diabetes Prediction Model - 2

## Importing the dependencies

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

## Loading the data

In [2]:
diabetes_data = pd.read_csv('diabetes.csv')

## Exploring the data and eliminating class imbalance

In [3]:
diabetes_data.shape

(768, 9)

In [4]:
diabetes_data.sample(5)

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
349,5,0,80,32,0,41.0,0.346,37,1
693,7,129,68,49,125,38.5,0.439,43,1
190,3,111,62,0,0,22.6,0.142,21,0
509,8,120,78,0,0,25.0,0.409,64,0
513,2,91,62,0,0,27.3,0.525,22,0


In [5]:
diabetes_data['Outcome'].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [7]:
diabetes_positive = diabetes_data[diabetes_data['Outcome'] == 1]
diabetes_negative = diabetes_data[diabetes_data['Outcome'] == 0]

In [8]:
print(diabetes_positive.shape, diabetes_negative.shape)

(268, 9) (500, 9)


In [10]:
diabetes_negative_sample = diabetes_negative.sample(diabetes_positive.shape[0])

In [14]:
diabetes_df = pd.concat([diabetes_negative_sample, diabetes_positive])

In [15]:
diabetes_df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
200,0,113,80,16,0,31.0,0.874,21,0
380,1,107,72,30,82,30.8,0.821,24,0
361,5,158,70,0,0,29.8,0.207,63,0
190,3,111,62,0,0,22.6,0.142,21,0
735,4,95,60,32,0,35.4,0.284,28,0
...,...,...,...,...,...,...,...,...,...
755,1,128,88,39,110,36.5,1.057,37,1
757,0,123,72,0,0,36.3,0.258,52,1
759,6,190,92,0,0,35.5,0.278,66,1
761,9,170,74,31,0,44.0,0.403,43,1


In [16]:
diabetes_df['Outcome'].value_counts()

0    268
1    268
Name: Outcome, dtype: int64

## Splitting the features and the label

In [17]:
X = diabetes_df.drop(columns=['Outcome'])
y = diabetes_df['Outcome']

## Splitting the data into training set and test set

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2, stratify=y)

## Feature scaling the models

In [19]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

## Training the models and comparing the accuracy scores

In [20]:
classifiers = [
    {
        'name': 'Logistic Regression',
        'classifier': LogisticRegression(max_iter=2000),
        'params': {
            'C': [0.1, 1.0, 10.0],
            'solver': ['liblinear', 'lbfgs']
        },
        'train_data': 'X_train_scaled'
    },
    {
        'name': 'Decision Tree',
        'classifier': DecisionTreeClassifier(),
        'params': {
            'max_depth': [None, 5, 10],
            'min_samples_split': [2, 5, 10]
        },
        'train_data': 'X_train'
    },
    {
        'name': 'Support Vector Machine',
        'classifier': SVC(max_iter=2000),
        'params': {
            'C': [0.1, 1.0, 10.0],
            'kernel': ['linear', 'rbf']
        },
        'train_data': 'X_train_scaled'
    },
    {
        'name': 'Random Forest',
        'classifier': RandomForestClassifier(),
        'params': {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 5, 10]
        },
        'train_data': 'X_train'
    },
    {
        'name': 'Gaussian Naive Bayes',
        'classifier': GaussianNB(),
        'params': {},
        'train_data': 'X_train'
    },
    {
        'name': 'XGB Classifer',
        'classifier': XGBClassifier(),
        'params': {},
        'train_data': 'X_train'
    },
    {
        'name': 'CatBoost Classifier',
        'classifier': CatBoostClassifier(),
        'params': {},
        'train_data': 'X_train'
    }
]


In [21]:
results_df = pd.DataFrame(columns=['Classifier', 'Best Parameters', 'Accuracy'])

In [22]:
for classifier in classifiers:
    if classifier['train_data'] == 'X_train':
        X_train_data = X_train
    elif classifier['train_data'] == 'X_train_scaled':
        X_train_data = X_train_scaled
    else:
        raise ValueError("Invalid train_data value. Must be 'X_train' or 'X_train_scaled'.")
    
    grid_search = GridSearchCV(classifier['classifier'], classifier['params'], cv=5)
    grid_search.fit(X_train_data, y_train)
    best_classifier = grid_search.best_estimator_
    if classifier['train_data'] == 'X_train':
        y_pred = best_classifier.predict(X_test)
    elif classifier['train_data'] == 'X_train_scaled':
        X_test_scaled = scaler.transform(X_test)
        y_pred = best_classifier.predict(X_test_scaled)
    
    accuracy = accuracy_score(y_test, y_pred)
    
    # Append the results to the DataFrame
    results_df = results_df.append({
        'Classifier': classifier['name'],
        'Best Parameters': grid_search.best_params_,
        'Accuracy': accuracy
    }, ignore_index=True)


C:\Users\anupa\AppData\Local\Temp\ipykernel_6908\4204547685.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\anupa\AppData\Local\Temp\ipykernel_6908\4204547685.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({
C:\Users\anupa\.android\trial\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\anupa\.android\trial\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=2000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
C:\Users\anupa\.android\trial\lib\site-packages\sklearn\svm\_base.py:284: Converge

Learning rate set to 0.006516
0:	learn: 0.6896140	total: 144ms	remaining: 2m 24s
1:	learn: 0.6859968	total: 148ms	remaining: 1m 13s
2:	learn: 0.6835738	total: 152ms	remaining: 50.5s
3:	learn: 0.6809309	total: 155ms	remaining: 38.7s
4:	learn: 0.6780149	total: 158ms	remaining: 31.4s
5:	learn: 0.6752317	total: 160ms	remaining: 26.6s
6:	learn: 0.6725740	total: 163ms	remaining: 23.1s
7:	learn: 0.6703587	total: 165ms	remaining: 20.5s
8:	learn: 0.6668378	total: 167ms	remaining: 18.4s
9:	learn: 0.6634585	total: 169ms	remaining: 16.7s
10:	learn: 0.6604131	total: 171ms	remaining: 15.4s
11:	learn: 0.6584056	total: 173ms	remaining: 14.3s
12:	learn: 0.6568991	total: 175ms	remaining: 13.3s
13:	learn: 0.6549047	total: 177ms	remaining: 12.5s
14:	learn: 0.6526750	total: 179ms	remaining: 11.7s
15:	learn: 0.6498843	total: 181ms	remaining: 11.1s
16:	learn: 0.6463642	total: 183ms	remaining: 10.6s
17:	learn: 0.6434679	total: 184ms	remaining: 10.1s
18:	learn: 0.6414853	total: 186ms	remaining: 9.62s
19:	learn

215:	learn: 0.4052302	total: 603ms	remaining: 2.19s
216:	learn: 0.4046304	total: 606ms	remaining: 2.19s
217:	learn: 0.4039361	total: 608ms	remaining: 2.18s
218:	learn: 0.4033668	total: 610ms	remaining: 2.18s
219:	learn: 0.4027989	total: 613ms	remaining: 2.17s
220:	learn: 0.4022912	total: 615ms	remaining: 2.17s
221:	learn: 0.4013181	total: 617ms	remaining: 2.16s
222:	learn: 0.4008419	total: 619ms	remaining: 2.16s
223:	learn: 0.4003161	total: 621ms	remaining: 2.15s
224:	learn: 0.3996685	total: 624ms	remaining: 2.15s
225:	learn: 0.3991758	total: 626ms	remaining: 2.14s
226:	learn: 0.3984515	total: 628ms	remaining: 2.14s
227:	learn: 0.3980524	total: 630ms	remaining: 2.13s
228:	learn: 0.3972830	total: 632ms	remaining: 2.13s
229:	learn: 0.3967242	total: 634ms	remaining: 2.12s
230:	learn: 0.3962849	total: 637ms	remaining: 2.12s
231:	learn: 0.3954745	total: 639ms	remaining: 2.11s
232:	learn: 0.3951081	total: 641ms	remaining: 2.11s
233:	learn: 0.3945346	total: 643ms	remaining: 2.1s
234:	learn: 0

404:	learn: 0.3229332	total: 992ms	remaining: 1.46s
405:	learn: 0.3227191	total: 994ms	remaining: 1.45s
406:	learn: 0.3225050	total: 996ms	remaining: 1.45s
407:	learn: 0.3223008	total: 998ms	remaining: 1.45s
408:	learn: 0.3219948	total: 1000ms	remaining: 1.44s
409:	learn: 0.3217894	total: 1s	remaining: 1.44s
410:	learn: 0.3215459	total: 1s	remaining: 1.44s
411:	learn: 0.3211746	total: 1s	remaining: 1.44s
412:	learn: 0.3210007	total: 1.01s	remaining: 1.43s
413:	learn: 0.3206918	total: 1.01s	remaining: 1.43s
414:	learn: 0.3204827	total: 1.01s	remaining: 1.43s
415:	learn: 0.3199165	total: 1.01s	remaining: 1.42s
416:	learn: 0.3196908	total: 1.02s	remaining: 1.42s
417:	learn: 0.3195281	total: 1.02s	remaining: 1.42s
418:	learn: 0.3193008	total: 1.02s	remaining: 1.41s
419:	learn: 0.3190261	total: 1.02s	remaining: 1.41s
420:	learn: 0.3185649	total: 1.02s	remaining: 1.41s
421:	learn: 0.3182869	total: 1.02s	remaining: 1.41s
422:	learn: 0.3177561	total: 1.03s	remaining: 1.4s
423:	learn: 0.3174587

604:	learn: 0.2683534	total: 1.38s	remaining: 904ms
605:	learn: 0.2680511	total: 1.39s	remaining: 902ms
606:	learn: 0.2679186	total: 1.39s	remaining: 900ms
607:	learn: 0.2677160	total: 1.39s	remaining: 898ms
608:	learn: 0.2675002	total: 1.39s	remaining: 895ms
609:	learn: 0.2673819	total: 1.4s	remaining: 893ms
610:	learn: 0.2672069	total: 1.4s	remaining: 890ms
611:	learn: 0.2669488	total: 1.4s	remaining: 888ms
612:	learn: 0.2666767	total: 1.4s	remaining: 886ms
613:	learn: 0.2664268	total: 1.41s	remaining: 884ms
614:	learn: 0.2661997	total: 1.41s	remaining: 881ms
615:	learn: 0.2658523	total: 1.41s	remaining: 879ms
616:	learn: 0.2657565	total: 1.41s	remaining: 876ms
617:	learn: 0.2655712	total: 1.41s	remaining: 874ms
618:	learn: 0.2653134	total: 1.42s	remaining: 871ms
619:	learn: 0.2649652	total: 1.42s	remaining: 869ms
620:	learn: 0.2646927	total: 1.42s	remaining: 866ms
621:	learn: 0.2645478	total: 1.42s	remaining: 864ms
622:	learn: 0.2644292	total: 1.42s	remaining: 861ms
623:	learn: 0.26

...

357:	learn: 0.3572966	total: 815ms	remaining: 1.46s
358:	learn: 0.3569479	total: 818ms	remaining: 1.46s
359:	learn: 0.3567032	total: 821ms	remaining: 1.46s
360:	learn: 0.3564360	total: 824ms	remaining: 1.46s
361:	learn: 0.3563180	total: 826ms	remaining: 1.46s
362:	learn: 0.3560131	total: 828ms	remaining: 1.45s
363:	learn: 0.3556512	total: 830ms	remaining: 1.45s
364:	learn: 0.3552374	total: 832ms	remaining: 1.45s
365:	learn: 0.3547803	total: 835ms	remaining: 1.45s
366:	learn: 0.3544918	total: 837ms	remaining: 1.44s
367:	learn: 0.3541566	total: 839ms	remaining: 1.44s
368:	learn: 0.3536861	total: 842ms	remaining: 1.44s
369:	learn: 0.3533408	total: 843ms	remaining: 1.44s
370:	learn: 0.3529576	total: 845ms	remaining: 1.43s
371:	learn: 0.3525954	total: 847ms	remaining: 1.43s
372:	learn: 0.3523929	total: 849ms	remaining: 1.43s
373:	learn: 0.3521109	total: 851ms	remaining: 1.42s
374:	learn: 0.3515699	total: 853ms	remaining: 1.42s
375:	learn: 0.3512535	total: 856ms	remaining: 1.42s
376:	learn: 

540:	learn: 0.3057176	total: 1.19s	remaining: 1.01s
541:	learn: 0.3054873	total: 1.19s	remaining: 1s
542:	learn: 0.3051865	total: 1.19s	remaining: 1s
543:	learn: 0.3050181	total: 1.19s	remaining: 1s
544:	learn: 0.3046633	total: 1.2s	remaining: 999ms
545:	learn: 0.3044594	total: 1.2s	remaining: 997ms
546:	learn: 0.3041921	total: 1.2s	remaining: 994ms
547:	learn: 0.3040256	total: 1.2s	remaining: 992ms
548:	learn: 0.3038327	total: 1.21s	remaining: 990ms
549:	learn: 0.3035735	total: 1.21s	remaining: 988ms
550:	learn: 0.3032362	total: 1.21s	remaining: 985ms
551:	learn: 0.3029653	total: 1.21s	remaining: 983ms
552:	learn: 0.3027129	total: 1.21s	remaining: 981ms
553:	learn: 0.3026483	total: 1.21s	remaining: 978ms
554:	learn: 0.3023831	total: 1.22s	remaining: 976ms
555:	learn: 0.3022871	total: 1.22s	remaining: 973ms
556:	learn: 0.3019231	total: 1.22s	remaining: 971ms
557:	learn: 0.3015942	total: 1.22s	remaining: 968ms
558:	learn: 0.3013952	total: 1.22s	remaining: 966ms
559:	learn: 0.3011977	tot

750:	learn: 0.2618394	total: 1.59s	remaining: 527ms
751:	learn: 0.2613133	total: 1.59s	remaining: 525ms
752:	learn: 0.2609813	total: 1.59s	remaining: 523ms
753:	learn: 0.2606222	total: 1.6s	remaining: 521ms
754:	learn: 0.2604977	total: 1.6s	remaining: 519ms
755:	learn: 0.2602703	total: 1.6s	remaining: 517ms
756:	learn: 0.2600797	total: 1.6s	remaining: 515ms
757:	learn: 0.2599492	total: 1.6s	remaining: 513ms
758:	learn: 0.2597108	total: 1.61s	remaining: 511ms
759:	learn: 0.2595909	total: 1.61s	remaining: 508ms
760:	learn: 0.2594212	total: 1.61s	remaining: 506ms
761:	learn: 0.2592226	total: 1.61s	remaining: 504ms
762:	learn: 0.2590362	total: 1.62s	remaining: 502ms
763:	learn: 0.2589511	total: 1.62s	remaining: 500ms
764:	learn: 0.2587747	total: 1.62s	remaining: 498ms
765:	learn: 0.2584685	total: 1.62s	remaining: 496ms
766:	learn: 0.2583622	total: 1.62s	remaining: 493ms
767:	learn: 0.2581796	total: 1.63s	remaining: 491ms
768:	learn: 0.2580812	total: 1.63s	remaining: 489ms
769:	learn: 0.257

928:	learn: 0.2286883	total: 1.97s	remaining: 150ms
929:	learn: 0.2286030	total: 1.97s	remaining: 148ms
930:	learn: 0.2284552	total: 1.97s	remaining: 146ms
931:	learn: 0.2284012	total: 1.97s	remaining: 144ms
932:	learn: 0.2282157	total: 1.98s	remaining: 142ms
933:	learn: 0.2280571	total: 1.98s	remaining: 140ms
934:	learn: 0.2277663	total: 1.98s	remaining: 138ms
935:	learn: 0.2275209	total: 1.98s	remaining: 136ms
936:	learn: 0.2271448	total: 1.98s	remaining: 133ms
937:	learn: 0.2271113	total: 1.99s	remaining: 131ms
938:	learn: 0.2268980	total: 1.99s	remaining: 129ms
939:	learn: 0.2266539	total: 1.99s	remaining: 127ms
940:	learn: 0.2264842	total: 1.99s	remaining: 125ms
941:	learn: 0.2263970	total: 2s	remaining: 123ms
942:	learn: 0.2262814	total: 2s	remaining: 121ms
943:	learn: 0.2260979	total: 2s	remaining: 119ms
944:	learn: 0.2259144	total: 2s	remaining: 116ms
945:	learn: 0.2258180	total: 2s	remaining: 114ms
946:	learn: 0.2256993	total: 2s	remaining: 112ms
947:	learn: 0.2256481	total: 2

C:\Users\anupa\AppData\Local\Temp\ipykernel_6908\4204547685.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results_df = results_df.append({


In [23]:
results_df = results_df.sort_values(by='Accuracy', ascending=False)

In [24]:
results_df

,Classifier,Best Parameters,Accuracy
3,Random Forest,"{'max_depth': 5, 'n_estimators': 200}",0.777778
0,Logistic Regression,"{'C': 1.0, 'solver': 'liblinear'}",0.750000
4,Gaussian Naive Bayes,{},0.740741
6,CatBoost Classifier,{},0.740741
2,Support Vector Machine,"{'C': 1.0, 'kernel': 'rbf'}",0.731481
5,XGB Classifer,{},0.731481
1,Decision Tree,"{'max_depth': 5, 'min_samples_split': 2}",0.685185


Eliminating class imbalance didn't turn out to be good for most models except Random Forest Classifier. Even that doesn't have the accuracy I would have liked it to. It isn't even 80%